### Automated Hyperparameter Tunning
    -> Bayyesian Optimization
    -> Gradient Descent 
    -> Evolutionary Algorithms
### Bayesian Optimization

    Bayesian approaches, in contrast to random or grid search, keep track of past evaluation results which they 
    use to form a probabilistic model mapping hyperparameters to a probability of a score on the objective 
    function.
                        p(score/hyperparameters)
                        
    Bayesian optimization uses probability to find the minimum of a function.The final aim is to find the input
    value to a function which can give us the lowest possible output value.it usually performs better than 
    random,grid and manual search providing betters performance in the testing phase and reduced optimization
    time.In otimization can be implemented giving three main parameters to the function fmin.
        
            1. Objective function = defines the loss function to minimize.
            2. Domain space = defines the range of input values to test (in bayesian optimization this space
            creates a probability distribution for each of the used hyperparameters)
            3. Optimization Algritham = defines the search algorithm to use to select the best input values to 
            use in each iteration
            
   

In [22]:
import pandas as pd 
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import warnings
warnings.filterwarnings('ignore')

In [23]:
df=pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [24]:
df['Glucose']=np.where(df['Glucose']==0,df['Glucose'].median(),df['Glucose'])
df['Insulin']=np.where(df['Insulin']==0,df['Insulin'].median(),df['Insulin'])
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72,35,30.5,33.6,0.627,50,1
1,1,85.0,66,29,30.5,26.6,0.351,31,0
2,8,183.0,64,0,30.5,23.3,0.672,32,1
3,1,89.0,66,23,94.0,28.1,0.167,21,0
4,0,137.0,40,35,168.0,43.1,2.288,33,1


In [25]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [26]:
from sklearn.model_selection import train_test_split
x_tain,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=33)

In [27]:
x_tain.shape,y_train.shape,x_test.shape,y_test.shape

((614, 8), (614,), (154, 8), (154,))

In [28]:
## using hyperopt hyperparameter tunning
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

In [29]:
space={'criterion':hp.choice('criterion',['entropy','gini']),
      'max_depth':hp.quniform('max_depth',10,1200,10),
      'max_features':hp.choice('max_features',['auto','sqrt','log2',None]),
      'min_samples_leaf': hp.uniform('min_samples_leaf',0, 0.5),
      'min_samples_split':hp.uniform('min_samples_split',0,1),
      'n_estimators':hp.choice('n_estimators',[10,50,100,150,300,700,1200,1300])}

In [30]:
space

{'criterion': <hyperopt.pyll.base.Apply at 0x7f0e680d9400>,
 'max_depth': <hyperopt.pyll.base.Apply at 0x7f0e680d95f8>,
 'max_features': <hyperopt.pyll.base.Apply at 0x7f0e680d9710>,
 'min_samples_leaf': <hyperopt.pyll.base.Apply at 0x7f0e680d9940>,
 'min_samples_split': <hyperopt.pyll.base.Apply at 0x7f0e680d9a90>,
 'n_estimators': <hyperopt.pyll.base.Apply at 0x7f0e680d9ba8>}

In [31]:
def objective(space):
    model=RandomForestClassifier(criterion=space['criterion'],max_depth=space['max_depth'],
                                max_features=space['max_features'],min_samples_leaf=space['min_samples_leaf'],
                                min_impurity_split=space['min_samples_split'],
                                n_estimators=space['n_estimators'])
    accuracy=cross_val_score(model,x_tain,y_train,cv=5).mean()
    return { 'loss': -accuracy,'status':STATUS_OK}

In [ ]:
trials=Trials()
best=fmin(fn=objective,
     space=space,
     algo=tpe.suggest,
     max_evals=80,
     trials=trials)
best

 98%|█████████▊| 78/80 [03:02<00:06,  3.41s/trial, best loss: -0.773610555777689] 

In [ ]:
crit={0:'entropy',1:'gini'}
feat={0:'auto',1:'sqrt',2:'log2',3:None}
est={0:10,1:50,2:300,3:750,4:1200,5:1300,6:1500}
print(crit[best['criterion']])
print(feat[best['max_features']])
print(est[best['n_estimators']])

In [ ]:
best['min_samples_leaf']

In [21]:
train_forest=RandomForestClassifier(criterion=crit[best['criterion']],
                                   max_depth=best['max_depth'],
                                   max_features=feat[best['max_features']],
                                   min_samples_leaf=best['min_samples_leaf'],
                                   min_samples_split=best['min_samples_split'],
                                   n_estimators=est[best['n_estimators']]).fit(x_tain,y_train)
prediction_rf=train_forest.predict(x_test)
print(confusion_matrix(y_test,prediction_rf))
print(accuracy_score(y_test,prediction_rf))
print(classification_report(y_test,prediction_rf))
acc=accuracy_score(y_test,prediction_rf)

[[99  0]
 [55  0]]
0.6428571428571429
              precision    recall  f1-score   support

           0       0.64      1.00      0.78        99
           1       0.00      0.00      0.00        55

    accuracy                           0.64       154
   macro avg       0.32      0.50      0.39       154
weighted avg       0.41      0.64      0.50       154

